In [1]:
import os

Reading API Keys

In [2]:
#Reading OpenAI Key
OPENAI_API_KEY=input("Enter OpenAI API Key")
#Reading Langchain API Key
LANGCHAIN_API_KEY=input("Enter Langchain API Key")

In [3]:
#Enabiling Langchain Tracing
LANGCHAIN_TRACING_V2= "true"
#OpenAI Model
OPENAI_MODEL="gpt-4o-mini"
LANGCHAIN_TRACING_V2="true"


In [ ]:
# installing langchain_openai package
! pip install langchain_openai

In [4]:
# initializing llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model=OPENAI_MODEL,openai_api_key=OPENAI_API_KEY,temperature=0.5)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10fed5940>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10fed6ed0>, root_client=<openai.OpenAI object at 0x10f5bc7a0>, root_async_client=<openai.AsyncOpenAI object at 0x10fed5970>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
# Installing Python package for SQLDatabase
! pip install langchain-community==0.2.9
! pip install sqlalchemy==2.0.28

In [5]:
#initializing Database object
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///baggage_tracking.db")
db

In [6]:
from langchain_core.output_parsers import BaseGenerationOutputParser
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.outputs import  Generation
class SQlExtractParser(BaseGenerationOutputParser[str]):
    """
    A parser for SQL output.
    """

    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """
        Parse the SQL query input and return the extracted SQL query.

        Args:
            sql_query_input (str): The input SQL query string.
            partial (bool, optional): Whether the input is a partial query. Defaults to False.

        Returns:
            str: The extracted SQL query.
        """
        if not result:
            raise OutputParserException("Input SQL query is empty")
        
        print(f"Incoming query {result[0]}")

        if 'SQLQuery:' in result[0].text:
            sql_query = result[0].text.split('SQLQuery:', 1)[1].strip()
            print(f"Extracted SQL query: {sql_query}")
        else:
            sql_query = result[0].text

        return sql_query

In [7]:
sql_examples = [
    {
        "input": "Show bag tag numbers for the PNR number AAA100",
        "query": "select bag_tag_number from baggage_details where pnr = 'AAA100';"
    },
    {
        "input": "Find the baggage details of the flight originating from JFK and destination is SFO",
        "query": "select * from baggage_details where origin ='JFK' and destination ='SFO';"
    },
    {
        "input": "Find scanning details of bag tag number 001234567890",
        "query": "SELECT * from baggage_scanning where bag_tag_number = '001234567890';"
    },
    {
        "input": "Find the passengers in flight BC101",
        "query": "SELECT * from passenger_details where passenger_flight_number = 'BC101';"
    },
    {
        "input": "Find the passenger details for the bag tag number 001234567890",
        "query": "SELECT * from passenger_details where bag_tag_number = '001234567890'"
    },
    {
        "input": "Find rush bags in flight AB100",
        "query": "SELECT * from baggage_details where rush_bag='Yes' and flight_number = 'AB100'"
    }
]

Langchain Example  Selector
=====================================
langchain_core.example_selectors is a module in LangChain that deals with selecting examples dynamically from a set of predefined examples based on the current input or prompt. This is useful when you want to choose the most relevant examples to show or use for few-shot learning, helping to improve the performance of large language models (LLMs) by providing them with more specific and contextual examples.

Example Selectors dynamically pick the most relevant examples from a set, based on the input to the language model. Instead of hardcoding examples for few-shot learning, example selectors allow you to choose relevant examples based on input.



In [ ]:
! pip install langchain_chroma

In [8]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
# initializes an instance of a Chroma vector store.
vector_store = Chroma()
# deletes any existing collection in the vector store
vector_store.delete_collection()
#  LangChain class that helps you select examples based on their semantic similarity to a given input. 
# It uses vector embeddings (vectorized representations of text) to compare the input with examples.
# creating creates an instance of SemanticSimilarityExampleSelector
sql_example_selector = SemanticSimilarityExampleSelector.from_examples(
    sql_examples,
    # To transform SQL examples and input queries into embeddings (vectors), enabling semantic comparison between them.
    OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    # This is the Chroma vector store, where embeddings of the examples will be stored and queried. 
    vector_store,
    # This specifies that the selector should return the top 2 most similar examples. 
    k=2,
    #  This indicates that the key "input" in the provided examples will be used as the query. 
    # It tells the selector what part of each example should be compared to the input query.
    input_keys=["input"]
)

In [9]:
from langchain_core.prompts import ChatPromptTemplate
sql_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai","{query}")
    ]
)

In [10]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
sql_few_shot_prompt = FewShotChatMessagePromptTemplate(
    # This is the format/template used for each individual example in the few-shot learning setup. 
    # It defines how each example (SQL query and its corresponding result) will be presented.
    example_prompt=sql_example_prompt,
    # example selector that dynamically chooses the most relevant examples based on the user's input.
    example_selector=sql_example_selector,
    # This indicates which variables (placeholders) from the input will be used in the prompt template.
    input_variables=["input"]
)

In [39]:
from langchain_core.prompts import  ChatPromptTemplate,MessagesPlaceholder
SQL_QUERY_PROMPT_WITH_MEMORY=ChatPromptTemplate.from_messages(
    [
        ("system",""" You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
         You can order the results to return the most informative data in the database.
        Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
        Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
        Pay attention to use date('now') function to get the current date, if the question involves "today".

        Use the following format:

        Question: Question here
        SQLQuery: SQL Query to run
        SQLResult: Result of the SQLQuery
        Answer: Final answer here
         
        {top_k}

        Only use the following tables:
        {table_info}"""),
        sql_few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human","{input}")
    ]
)

In [12]:
from langchain_core.prompts import PromptTemplate
ANSWER_PROMPTS = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
     Combine the the Question and SQL Result to generate a meaningful answer to the user.
     The answer should be in the format of a sentence.
     The answer should be a direct response to the user question.
     Respond to the question as an experienced airline ground staff

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

Dynamic Table Selection
========================

We are making use of the table_names_to_use parameter in create_sql_query_chain
If the user specifies "table_names_to_use" when invoking chain, only those will be included. Otherwise, all tables are included.


In [13]:
from langchain_core.prompts import ChatPromptTemplate
TABLE_SELECTION_PROMPT=ChatPromptTemplate.from_messages(
    [
        ("system","""Return the names of ALL the SQL tables that MIGHT be relevant to the user question.
        The tables are:
        {table_names}
        Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.
        Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
        """),
        ("human","{input}")
    ]
)

In [14]:
from operator import itemgetter
from langchain.output_parsers import CommaSeparatedListOutputParser
table_selection_chain={"input":itemgetter("question"),"table_names":itemgetter("table_names")}|TABLE_SELECTION_PROMPT|llm|CommaSeparatedListOutputParser()

In [ ]:
! pip install pandas

In [15]:
import pandas as pd 
table_details = pd.read_csv("./table_details.csv")

table_description_array = []
for index, row in table_details.iterrows():
    table_description_array.append("Table Name:" + row['TableName'] + "\n" + "Table Description:" + row['Description'] + "\n\n")

table_description = '\n'.join(table_description_array)
print("Table description ", table_description)

Table description  Table Name:baggage_details
Table Description:This table stores the high-level details of each piece of baggage.  This table provides a comprehensive overview of each piece of baggage, useful for tracking and auditing purposes.


Table Name:baggage_scanning
Table Description:This table records the scanning details of each bag at various points in the journey. This table is crucial for monitoring the real-time status and movement of baggage, enabling proactive issue resolution and customer service.


Table Name:passenger_details
Table Description:This table stores information about the passengers traveling with the bags. This table allows airlines to associate passengers with their baggage, which is essential for handling rush bags where passengers and their bags might travel on different flights. It also facilitates personalized service and handling of lost baggage claims.




In [44]:
table_selection_chain.invoke({"question":"List all the flight names?",
                             "table_names":table_description})

['baggage_details', 'baggage_scanning', 'passenger_details']

In [ ]:
! pip install boto3

In [48]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_community.chat_message_histories.dynamodb import DynamoDBChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# Create a query chain
query_chain = create_sql_query_chain(llm, db,prompt=SQL_QUERY_PROMPT_WITH_MEMORY) | SQlExtractParser()

current_session_id="Hello1"

# Create an execution chain
execute_query = QuerySQLDataBaseTool(db=db)
execute_chain = itemgetter("query") | execute_query

# Create an answer chain
answer_chain = ANSWER_PROMPTS | llm | StrOutputParser()

# Combine the chains
final_chain = (
    RunnablePassthrough()
    .assign(table_names_to_use=table_selection_chain)
    .assign(query=query_chain)
    .assign(result=execute_chain)
    | answer_chain
)

# Invoke the chain with the user's query
input_data = {
    "question": "Which flight has the passenger John Smith travelled?",
    "dialect": "sqlite",
    "top_k":2,
    "table_names": table_description,
}

chain_with_history = RunnableWithMessageHistory(
    final_chain,
    lambda session_id: DynamoDBChatMessageHistory(
        table_name="SessionTable", 
        session_id=session_id,
        history_size=10
        ),
        input_messages_key="question",
        history_messages_key="messages",
)
config = {
    "configurable": {"session_id": current_session_id}
    }
answer=chain_with_history.invoke(input_data, config=config)



Incoming query text='SELECT "passenger_flight_number" FROM "passenger_details" WHERE "passenger_name" = \'John Smith\';'


In [49]:
answer

'John Smith has traveled on flight BC101.'

In [52]:
input_data = {
    "question": "from where the fligt originated",
    "dialect": "sqlite",
    "top_k":2,
    "table_names": table_description,
}
answer=chain_with_history.invoke(input_data, config=config)

Incoming query text='SELECT "origin" FROM baggage_details WHERE "pnr" = \'AAA100\';'


In [53]:
answer

'The flight originated from JFK.'

In [56]:
input_data = {
    "question": "What is the flight destination",
    "dialect": "sqlite",
    "top_k":2,
    "table_names": table_description,
}
answer=chain_with_history.invoke(input_data, config=config)

Incoming query text='SELECT "destination" FROM baggage_details WHERE "pnr" = \'AAA100\';'


In [57]:
answer

'The flight destination is San Francisco (SFO).'

In [58]:
input_data = {
    "question": "List his bag tag numbers",
    "dialect": "sqlite",
    "top_k":2,
    "table_names": table_description,
}
answer=chain_with_history.invoke(input_data, config=config)

Incoming query text='SQLQuery: SELECT "bag_tag_number" FROM passenger_details WHERE "passenger_name" = \'John Smith\';'
Extracted SQL query: SELECT "bag_tag_number" FROM passenger_details WHERE "passenger_name" = 'John Smith';


In [59]:
answer

'Your bag tag number is 001234567890, John Smith.'